In [1]:
!pip install -q snscrape==0.3.4

In [2]:
pip install emoji

     |████████████████████████████████| 174 kB 4.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=30210a138baf7b35ec84956720c06b2fd80820dffd4ea13ff7b967a260e20119
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [3]:
import os
import pandas as pd
from datetime import date

In [4]:
today = date.today()
end_date = today

In [5]:
from_date = '2021-01-01' 
# Only uptop 1000 search will fetch
max_results = 1000
# List of item that need to be searched
search_term = ['aap','bsp','bjp','sp','congress']

In [6]:
'''
This will search the listed item in the above mentoined list from twitter
It will also create the individual file with the name of listed item from above mentioned each
'''
for i in range(len(search_term)):
  os.system("snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term[i]} until:{end_date}' > {search_term[i]}.txt")

In [7]:
#This code will combine all those extracted file in single file named Combo.txt
for i in range(len(search_term)):
  with open("Combo.txt",'a') as f:
    data=open(f'{search_term[i]}.txt','r')
    containt=data.read()
    data.close()
    f.write(containt)

In [8]:
# This function is used to remove all the punctuation from the line
import string
def remove_pun(line):
  txt_nopun="".join([c for c in line if c not in string.punctuation])
  return txt_nopun

In [9]:
import re
import emoji
import csv
f=open("Combo.txt")
f1=open('extracted_tweets.txt','a')
f2=open('clean.txt','a')
f1.write("text\n")
for line in f:
  #Removes the username 
  line=re.sub('@[\w]+',"",line)
  # This function removes the all the emoji
  def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)
  # Remove the emoji
  line=remove_emoji(line)
  # Remove the punctuation
  line=remove_pun(line)
  # Writing to extracted_tweets.txt file
  f1.write(line)
  f2.write(line)
f.close()
f1.close()
# Creating the CSV file for clean data
df = pd.read_fwf('extracted_tweets.txt')
df.to_csv('extracted_tweets.csv',header=True,index=None)
f2.close()

In [10]:
df = pd.read_csv('extracted_tweets.csv')
list_name = ['aap','bsp','bjp','sp','congress'] 
storage = {i:0 for i in list_name}
print(storage)
n = len(df['text'])
print(n)
ans=[]
for i in range(n):

    val=True
    data= df['text'].iloc[i]
    data = str(data).lower()
    for j in list_name:
        if j in data:
            storage[j]+=1
            val = False
            
    if val:
        ans.append(0)
    else:
        ans.append(1)

final_storage=dict()
for i in storage.keys():
    if storage[i]!=0:
        final_storage[i]=storage[i]
print(final_storage)
df['label'] = ans
        
df.head(10)
df.to_csv('label.csv')
df

{'aap': 0, 'bsp': 0, 'bjp': 0, 'sp': 0, 'congress': 0}
4961
{'aap': 1065, 'bsp': 651, 'bjp': 1233, 'sp': 1276, 'congress': 1165}


,text,label
0,Aap ko bhi Ram Ram ji,1
1,भँवरे ने खिलाया फूलफूल को ले गया राजकुमार। किस...,1
2,109シャワーを浴びるu3000อาบน้ำ àap náamn【単語】n・浴びるu300...,0
3,Its also why the AAP recommends masking in sch...,1
4,Punjab EXIT POLL भाजपा और कांग्रेस की लड़ाई मे...,1
...,...,...
4956,Well I cant wait until you are defeated and no...,1
4957,Alfred E Neumann is a moron,0
4958,Congress will always remain a subversive antin...,1
4959,When the Republicans gain control of Congress ...,1


In [11]:
df["label"].value_counts()

1    3655
0    1306
Name: label, dtype: int64

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4961 non-null   object
 1   label   4961 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 77.6+ KB
None
